<a href="https://colab.research.google.com/github/teju336/TejaswiniSomasundar.github.io/blob/master/CORONASENTIMENTNEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import tweepy as tw
import pandas as pd
import csv
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials
import nltk
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split # function for splitting data to train and test sets
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

from subprocess import check_output
import numpy as np
import pandas as pd
import re
import warnings

#Visualisation
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from IPython.display import display
from wordcloud import WordCloud, STOPWORDS

#nltk
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize

matplotlib.style.use('ggplot')
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore")

In [15]:
  >>> import nltk
  >>> nltk.download('wordnet')
  >>> nltk.download('vader_lexicon')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [0]:
from google.colab import files
uploaded = files.upload()

Saving dfnew.csv to dfnew.csv


In [0]:
import io
df = pd.read_csv(io.BytesIO(uploaded['dfnew.csv']))

In [0]:
df['text'][1]

'RT @TV_Publica: #Coronavirus: El Gobierno nacional analiza extender la suspensión de clases. El ministro de Educación @trottanico resaltó l…'

In [0]:
def wordcloud(df,col):
    stopwords = set(STOPWORDS)
    wordcloud = WordCloud(background_color="white",stopwords=stopwords,random_state = 2009).generate(" ".join([i for i in df[col]]))
    plt.figure( figsize=(40,20), facecolor='k')
    plt.imshow(wordcloud)
    plt.axis("off")
wordcloud(df,'text')

In [0]:
df['text_lem'] = [''.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in df['text']]       
vectorizer = TfidfVectorizer(max_df=0.5,max_features=10000,min_df=10,stop_words='english',use_idf=True)
X = vectorizer.fit_transform(df['text_lem'].str.upper())
sid = SentimentIntensityAnalyzer()
df['sentiment_compound_polarity']=df.text_lem.apply(lambda x:sid.polarity_scores(x)['compound'])
df['sentiment_neutral']=df.text_lem.apply(lambda x:sid.polarity_scores(x)['neu'])
df['sentiment_negative']=df.text_lem.apply(lambda x:sid.polarity_scores(x)['neg'])
df['sentiment_pos']=df.text_lem.apply(lambda x:sid.polarity_scores(x)['pos'])
df['sentiment_type']=''
df.loc[df.sentiment_compound_polarity>0,'sentiment_type']='POSITIVE'
df.loc[df.sentiment_compound_polarity==0,'sentiment_type']='NEUTRAL'
df.loc[df.sentiment_compound_polarity<0,'sentiment_type']='NEGATIVE'

In [0]:
tweets_sentiment = df.groupby(['sentiment_type'])['sentiment_neutral'].count()
tweets_sentiment.rename("",inplace=True)
explode = (1, 0, 0)
plt.subplot(221)
tweets_sentiment.transpose().plot(kind='barh',figsize=(20, 20))
plt.title('Sentiment Analysis 1', bbox={'facecolor':'0.8', 'pad':0})
plt.subplot(222)
tweets_sentiment.plot(kind='pie',figsize=(20, 20),autopct='%1.1f%%',shadow=True,explode=explode)
plt.legend(bbox_to_anchor=(1, 1), loc=3, borderaxespad=0.)
plt.title('Sentiment Analysis 2', bbox={'facecolor':'0.8', 'pad':0})